In [4]:
import pandas as pd
import numpy as np
import json
import geopandas as gp
import h3
from placekey.api import PlacekeyAPI

%load_ext autotime

placekey_api_key = "NDd0VsiPKznvm89NLNj9DHrtU3GVzZ1h" 
pk_api = PlacekeyAPI(placekey_api_key)

attom_cols = ['[ATTOM ID]', 'SitusStateCode', 'AreaBuilding', 'AreaBuildingDefinitionCode', 'AreaLotSF', 'BuildingsCount', 'CompanyFlag', 'ContactOwnerMailAddressFull', 'ContactOwnerMailAddressInfoFormat', 'GeoQuality', 'LegalDescription', 'OwnerTypeDescription1', 'ParcelNumberRaw', 'PartyOwner1NameFull', 'PartyOwner2NameFull', 'PartyOwner3NameFull', 'PropertyAddressCity', 'PropertyAddressFull', 'PropertyAddressHouseNumber', 'PropertyAddressState', 'PropertyAddressStreetDirection', 'PropertyAddressStreetName', 'PropertyAddressStreetPostDirection', 'PropertyAddressStreetSuffix', 'PropertyAddressUnitPrefix', 'PropertyAddressUnitValue', 'PropertyAddressZIP', 'PropertyAddressZIP4', 'PropertyLatitude', 'PropertyLongitude', 'PropertyUseGroup', 'PropertyUseStandardized', 'RoomsCount', 'StatusOwnerOccupiedFlag', 'StoriesCount', 'UnitsCount', 'TaxMarketValueYear', 'TaxMarketValueImprovements', 'YearBuilt']
regrid_cols = ['ll_uuid', 'city', 'county', 'address', 'owner', 'owner2', 'legaldesc', 'parcelnumb', 'lbcs_activity', 'lbcs_function', 'lbcs_ownership', 'lbcs_site', 'lbcs_structure', 'll_bldg_count', 'll_bldg_footprint_sqft', 'll_gisacre', 
               # 'gisacre',  # SOME COUNTIES DON'T PROVIDE THIS
               'usps_vacancy', 'rdi', 'saledate', 'saleprice', 'dpv_codes', 'dpv_notes', 'dpv_status', 'dpv_type', 'saddno', 'saddpref', 'saddstr', 'saddstsuf', 'saddsttyp', 'scity', 'sunit', 'szip', 'lat', 'lon', 
               # 'geometry' # geometry is a different object from the properties object
              ]

SF52 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '72': 'PR'}
SF52R = {value : key for (key, value) in SF52.items()}
# NO PUERTO RICO
SF51 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY'}
SF51R = {value : key for (key, value) in SF51.items()}


/home/nhat/anaconda3/envs/py39/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.0-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


time: 950 µs (started: 2022-04-21 18:06:51 -04:00)


In [4]:
# from glob import glob
# lists = glob("/home/nhat/regrid-bucket/mo_*.ndgeojson.gz")
# lists = [k.split('/')[-1].split('.')[0] for k in lists]
# print(len(lists)) # 111 counties
# with open("temp_stems_mo.txt", 'w') as f:
#     for k in lists:
#         f.write(k + '\n')

111

# Uber H3 experiments

In [4]:
regrid = gp.read_file('/home/nhat/regrid-bucket/tx_loving.json')

In [6]:
def gen_h3_hex_vectors(lat, lon, res):
    return h3.geo_to_h3(lat, lon, res)
gen_h3_hex = np.vectorize(gen_h3_hex_vectors)

regrid[['lat', 'lon']] = regrid[['lat', 'lon']].astype(float)


In [7]:
res = 7
regrid[f'hex{res}'] = gen_h3_hex(regrid['lat'], regrid['lon'],  res)
regrid[f'hex{res}']

0       8748d409dffffff
1       87488924dffffff
2       87488924dffffff
3       8748d452bffffff
4       8748d4529ffffff
             ...       
2690    8748d4553ffffff
2691    8748d4470ffffff
2692    8748d4596ffffff
2693    8748d44a9ffffff
2694    8748d4418ffffff
Name: hex7, Length: 2695, dtype: object

In [11]:
# uber h3 hex code always have length = 15 = GEOID (censusblock)
len('8748d4470ffffff'), len('8848d409dbfffff'), len('8f48892cb245013')

(15, 15, 15)

# Check content of REGRID verse file

In [18]:
# Verse
verse = gp.read_file('/home/nhat/regrid-bucket/verse.csv.zip')
verse.shape

(3231, 19)

In [19]:
# verse.csv.zip: Missing last_refresh and date_added
verse[verse.filename_stem == 'va_rappahannock']

,id,county,state,seat,population,sqmi,geoid,city,table_name,total_objects,last_refresh,path,canonical_path,filename_stem,date_added,shapefile_size_flag,assessor_data_date,usps_data_date,geometry
1133,2926,Rappahannock,VA,Washington,7378,267,51157,,,,,/us/va/rappahannock,,va_rappahannock,,,,,None


In [20]:
verse = verse.replace(r'^\s*$', np.nan, regex=True)

In [29]:
verse = verse[['id', 'county', 'state', 'geoid', 'last_refresh', 
               'filename_stem', 'date_added', 'table_name']]
missing_refresh = verse[verse[['last_refresh']].isnull().any(axis=1)]
missing_refresh.shape
missing_refresh.isnull().sum()

id                 0
county             0
state              0
geoid              0
last_refresh     121
filename_stem      0
date_added       121
table_name       121
dtype: int64

In [28]:
missing_table = verse[verse[['table_name']].isnull().any(axis=1)]
missing_table.isnull().sum()

id                 0
county             0
state              0
geoid              0
last_refresh     121
filename_stem      0
date_added       121
table_name       121
dtype: int64

In [35]:
missing = verse[verse[['last_refresh', 'date_added']].isnull().all(axis=1)]
missing.to_csv('verse_missing.csv', index=False)

In [37]:
missing[missing.state == 'VA']

,id,county,state,geoid,last_refresh,filename_stem,date_added
1132,2877,Dickenson,VA,51051,NaN,va_dickenson,NaN
1133,2926,Rappahannock,VA,51157,NaN,va_rappahannock,NaN
1198,2949,Buena Vista,VA,51530,NaN,va_buena_vista,NaN
2259,2874,Craig,VA,51045,NaN,va_craig,NaN


# Jupyter-lab NOHUP note: Unfinished codes would be run regardless of cut/remove the subsequent cells; unless interrupt kernel!

In [105]:
import time
x = 10
time.sleep(3)
x = 20
x


KeyboardInterrupt: 

time: 1.91 s (started: 2022-04-18 13:33:46 -04:00)


In [ ]:
x = 40
x

# PENDING: ATTOM TaxAssessor delta files (_003 series)

In [ ]:
attom = pd.read_feather('data/attom-feather/mo.ftr')
attom.rename(columns = {"[ATTOM ID]" : 'aid'}, inplace = True)
attom.shape

In [34]:
cols = list(set(attom) - set(['zorder', 'rorder']))

In [51]:
delta_mo = pd.DataFrame(df.query('SitusStateCode == "MO"'), columns=df.columns)
delta_mo.rename(columns = {"[ATTOM ID]" : 'aid'}, inplace = True)
delta_mo.shape

(93, 318)

In [52]:
old = attom.query("aid == '149047091'")[cols]
old

,SitusCounty,PropertyAddressStreetDirection,AssessorLastSaleAmount,AreaLotSF,StoriesCount,PropertyUseGroup,Fireplace,PropertyLatitude,aid,FireplaceCount,...,AssessorPriorSaleAmount,PropertyAddressStreetName,LegalDescription,UnitsCount,PartyOwner2NameFull,PreviousAssessedValue,PartyOwner1NameFull,DeedOwner1NameFull,StatusOwnerOccupiedFlag,RoomsGameFlag
470933,Bates,E,0,12600.00,1,Residential,None,38.098579,149047091,0,...,0,OLIVE,"LTS 7 & 8 (EXC E 10'), BLK 42, T C 1ST ADDN",1,None,5430,JORDAN FLETCHER,JORDAN FLETCHER,None,0


In [53]:
new = delta_mo.query("aid == '149047091'")[cols]
new

,SitusCounty,PropertyAddressStreetDirection,AssessorLastSaleAmount,AreaLotSF,StoriesCount,PropertyUseGroup,Fireplace,PropertyLatitude,aid,FireplaceCount,...,AssessorPriorSaleAmount,PropertyAddressStreetName,LegalDescription,UnitsCount,PartyOwner2NameFull,PreviousAssessedValue,PartyOwner1NameFull,DeedOwner1NameFull,StatusOwnerOccupiedFlag,RoomsGameFlag
79,Bates,E,0,12600.00,1,Residential,NaN,38.098579,149047091,0,...,0,OLIVE,"LTS 7 & 8 (EXC E 10'), BLK 42, T C 1ST ADDN",0,NaN,5990,RANDY THOMAS ELLIOTT,RANDY THOMAS ELLIOTT,1,0


In [13]:
overlaps = attom[attom.aid.isin(delta_mo.aid)].copy()
overlaps